Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in `1_notmnist.ipynb`.

In [3]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [4]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [5]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [19]:
batch_size = 128
graph = tf.Graph()
image_size2 = image_size * image_size

def get_logits(data_set, weights, biais):
    return tf.nn.softmax(tf.matmul(data_set, weights) + biais)

with graph.as_default():
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size2))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)    
    tf_test_dataset = tf.constant(test_dataset) 
    
    beta_regul = tf.placeholder(tf.float32)
    
    weights = tf.Variable(tf.truncated_normal([image_size2, num_labels]))
    biais = tf.Variable(tf.zeros([num_labels]))
    
    logits = tf.matmul(tf_train_dataset, weights) + biais
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits)) \
        + beta_regul* tf.nn.l2_loss(weights)
    
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = get_logits(tf_valid_dataset, weights, biais)
    test_prediction = get_logits(tf_test_dataset, weights, biais)    

In [20]:
num_steps = 3001

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
    
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)

        batch_data = train_dataset[offset: (batch_size + offset), :]
        batch_labels = train_labels[offset: (batch_size + offset), :]

        feed_dict = { tf_train_dataset: batch_data, tf_train_labels: batch_labels, beta_regul: 1e-3}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)

        if (step %500 == 0):
                print("Loss at step %d: %f" % (step,l))
                print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_data))
                print('Validation accuracy: %.1f%%' % accuracy(valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Loss at step 0: 18.944595
Training accuracy: 1.6%
Validation accuracy: 11.7%
Loss at step 500: 2.745388
Training accuracy: 3.9%
Validation accuracy: 76.7%
Loss at step 1000: 1.553520
Training accuracy: 2.3%
Validation accuracy: 78.5%
Loss at step 1500: 1.528104
Training accuracy: 2.3%
Validation accuracy: 79.6%
Loss at step 2000: 1.008913
Training accuracy: 2.3%
Validation accuracy: 81.3%
Loss at step 2500: 0.927184
Training accuracy: 3.1%
Validation accuracy: 82.5%
Loss at step 3000: 0.925713
Training accuracy: 5.5%
Validation accuracy: 82.6%
Test accuracy: 89.3%


## Neural Network

In [32]:
batch_size = 128
num_hidden_nodes = 1024

def get_matmul(data_set, weights, biais):
    return tf.matmul(data_set, weights) + biais

graph = tf.Graph()
with graph.as_default():
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size2))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    beta_regul = tf.placeholder(tf.float32)
    
    # hidden Layer 1
    weights1 = tf.Variable(tf.truncated_normal([image_size2, num_hidden_nodes]))
    biais1 = tf.Variable(tf.zeros([num_hidden_nodes]))
    train_lay1 = tf.nn.relu(get_matmul(tf_train_dataset, weights1, biais1))
    
    # Layer 2
    weights2= tf.Variable(tf.truncated_normal([num_hidden_nodes, num_labels]))
    biais2 = tf.Variable(tf.zeros([num_labels]))
    logits = get_matmul(train_lay1, weights2, biais2)
    
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=tf_train_labels)) \
        + beta_regul * ( tf.nn.l2_loss(weights1) + tf.nn.l2_loss(weights2))
        
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    train_prediction = tf.nn.softmax(logits)
    
    lay1_valid = tf.nn.relu(get_matmul(valid_dataset, weights1, biais1))
    valid_prediction = get_logits(lay1_valid, weights2, biais2)
    
    lay1_test = tf.nn.relu(get_matmul(test_dataset, weights1, biais1))
    test_prediction = get_logits(lay1_test, weights2, biais2)
    

In [34]:
num_steps = 3001
batch_size = 128

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        
        feed_dict = { tf_train_dataset: batch_data, tf_train_labels: batch_labels, beta_regul: 1e-3 }
        
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        
        if(step%500 == 0):
              print("offset: %d" % offset)
              print("Minibatch loss at step %d: %f" % (step, l))
              print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
              print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
offset: 0
Minibatch loss at step 0: 648.161926
Minibatch accuracy: 16.4%
Validation accuracy: 29.0%
offset: 64000
Minibatch loss at step 500: 200.911011
Minibatch accuracy: 78.1%
Validation accuracy: 81.3%
offset: 128000
Minibatch loss at step 1000: 114.281639
Minibatch accuracy: 79.7%
Validation accuracy: 82.1%
offset: 192000
Minibatch loss at step 1500: 68.893257
Minibatch accuracy: 82.0%
Validation accuracy: 83.5%
offset: 56128
Minibatch loss at step 2000: 41.819633
Minibatch accuracy: 85.9%
Validation accuracy: 84.7%
offset: 120128
Minibatch loss at step 2500: 25.263294
Minibatch accuracy: 87.5%
Validation accuracy: 86.6%
offset: 184128
Minibatch loss at step 3000: 15.584317
Minibatch accuracy: 84.4%
Validation accuracy: 87.5%
Test accuracy: 92.8%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [46]:
batch_size = 128
num_hidden_nodes = 1024

def get_matmul(data_set, weights, biais):
    return tf.matmul(data_set, weights) + biais

graph = tf.Graph()
with graph.as_default():
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size2))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    beta_regul = tf.placeholder(tf.float32)
    
    # hidden Layer 1
    weights1 = tf.Variable(tf.truncated_normal([image_size2, num_hidden_nodes]))
    biais1 = tf.Variable(tf.zeros([num_hidden_nodes]))
    train_lay1 = tf.nn.relu(get_matmul(tf_train_dataset, weights1, biais1))
    
    # Layer 2
    weights2= tf.Variable(tf.truncated_normal([num_hidden_nodes, num_labels]))
    biais2 = tf.Variable(tf.zeros([num_labels]))
    logits = get_matmul(train_lay1, weights2, biais2)
    
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=tf_train_labels)) \
        + beta_regul * ( tf.nn.l2_loss(weights1) + tf.nn.l2_loss(weights2))
        
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    train_prediction = tf.nn.softmax(logits)
    
    lay1_valid = tf.nn.relu(get_matmul(valid_dataset, weights1, biais1))
    valid_prediction = get_logits(lay1_valid, weights2, biais2)
    
    lay1_test = tf.nn.relu(get_matmul(test_dataset, weights1, biais1))
    test_prediction = get_logits(lay1_test, weights2, biais2)
    

In [47]:
num_steps = 101
num_batches = 3

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        # offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        offset = step % num_batches
        
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        
        feed_dict = { tf_train_dataset: batch_data, tf_train_labels: batch_labels, beta_regul: 1e-3 }
        
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        
        if(step%5 == 0):
              print("offset: %d" % offset)
              print("Minibatch loss at step %d: %f" % (step, l))
              print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
              print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
offset: 0
Minibatch loss at step 0: 682.016052
Minibatch accuracy: 4.7%
Validation accuracy: 30.4%
offset: 2
Minibatch loss at step 5: 365.127441
Minibatch accuracy: 81.2%
Validation accuracy: 59.8%
offset: 1
Minibatch loss at step 10: 311.693939
Minibatch accuracy: 100.0%
Validation accuracy: 62.6%
offset: 0
Minibatch loss at step 15: 310.139099
Minibatch accuracy: 100.0%
Validation accuracy: 62.6%
offset: 2
Minibatch loss at step 20: 308.592377
Minibatch accuracy: 100.0%
Validation accuracy: 62.6%
offset: 1
Minibatch loss at step 25: 307.052521
Minibatch accuracy: 100.0%
Validation accuracy: 62.6%
offset: 0
Minibatch loss at step 30: 305.520905
Minibatch accuracy: 100.0%
Validation accuracy: 62.6%
offset: 2
Minibatch loss at step 35: 303.996429
Minibatch accuracy: 100.0%
Validation accuracy: 62.6%
offset: 1
Minibatch loss at step 40: 302.480164
Minibatch accuracy: 100.0%
Validation accuracy: 62.6%
offset: 0
Minibatch loss at step 45: 300.970764
Minibatch accuracy: 100.0%


---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [44]:
batch_size = 128
num_hidden_nodes = 1024

def get_matmul(data_set, weights, biais):
    return tf.matmul(data_set, weights) + biais

graph = tf.Graph()
with graph.as_default():
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size2))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    beta_regul = tf.placeholder(tf.float32)
    
    # hidden Layer 1
    weights1 = tf.Variable(tf.truncated_normal([image_size2, num_hidden_nodes]))
    biais1 = tf.Variable(tf.zeros([num_hidden_nodes]))
    train_lay1 = tf.nn.relu(get_matmul(tf_train_dataset, weights1, biais1))
    drop1 = tf.nn.dropout(train_lay1, 0.5)
    
    # Layer 2
    weights2= tf.Variable(tf.truncated_normal([num_hidden_nodes, num_labels]))
    biais2 = tf.Variable(tf.zeros([num_labels]))
    logits = get_matmul(drop1, weights2, biais2)
    
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=tf_train_labels))
    #\
    #    + beta_regul * ( tf.nn.l2_loss(weights1) + tf.nn.l2_loss(weights2))
        
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    train_prediction = tf.nn.softmax(logits)
    
    lay1_valid = tf.nn.relu(get_matmul(valid_dataset, weights1, biais1))
    valid_prediction = get_logits(lay1_valid, weights2, biais2)
    
    lay1_test = tf.nn.relu(get_matmul(test_dataset, weights1, biais1))
    test_prediction = get_logits(lay1_test, weights2, biais2)
    

In [45]:
num_steps = 101
num_batches = 3

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        #offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        offset = step % num_batches
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        
        feed_dict = { tf_train_dataset: batch_data, tf_train_labels: batch_labels, beta_regul: 1e-3 }
        
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        
        if(step%5 == 0):
              print("offset: %d" % offset)
              print("Minibatch loss at step %d: %f" % (step, l))
              print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
              print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
offset: 0
Minibatch loss at step 0: 546.755798
Minibatch accuracy: 10.9%
Validation accuracy: 35.0%
offset: 2
Minibatch loss at step 5: 126.182274
Minibatch accuracy: 71.9%
Validation accuracy: 57.1%
offset: 1
Minibatch loss at step 10: 2.270122
Minibatch accuracy: 98.4%
Validation accuracy: 64.3%
offset: 0
Minibatch loss at step 15: 8.718610
Minibatch accuracy: 96.9%
Validation accuracy: 62.0%
offset: 2
Minibatch loss at step 20: 1.651996
Minibatch accuracy: 98.4%
Validation accuracy: 65.7%
offset: 1
Minibatch loss at step 25: 2.568307
Minibatch accuracy: 97.7%
Validation accuracy: 64.5%
offset: 0
Minibatch loss at step 30: 1.098736
Minibatch accuracy: 98.4%
Validation accuracy: 63.2%
offset: 2
Minibatch loss at step 35: 2.209643
Minibatch accuracy: 97.7%
Validation accuracy: 65.0%
offset: 1
Minibatch loss at step 40: 0.513458
Minibatch accuracy: 98.4%
Validation accuracy: 65.4%
offset: 0
Minibatch loss at step 45: 0.729473
Minibatch accuracy: 98.4%
Validation accuracy: 65

---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---
